In [1]:
import numpy as np
import pandas as pd
import os
from xgboost.sklearn import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# read from level file
level = pd.read_csv('../../dataset/data_xyz/provinceData/level.csv')
gdp = pd.read_csv('../../dataset/gdp_2021.csv')
population_density = pd.read_csv('../../dataset/population_density.csv')
population7th = pd.read_csv('../../dataset/population7th.csv')
aging_rate_2021 = pd.read_csv('../../dataset/aging_rate_2021.csv')
vaccinate = pd.read_csv('../../dataset/vaccinate/vaccinataion_china.csv')

In [3]:
def findAllFile(base):
    for root, ds, fs in os.walk(base):
        for f in fs:
            if f.endswith('.json'):
#                 yield f
                fullname = os.path.join(root, f)
                yield fullname,f

In [4]:
# load data from every province
province_days = []
province_names = []
base = '../../dataset/data_xyz/provinceData'
total_data = pd.DataFrame()
for i,f in findAllFile(base):
    # transform date
    data = pd.read_json(i)
    data = pd.json_normalize(data['data'])
    data['dateId'] = pd.to_datetime(data['dateId'], format='%Y%m%d')
    # transform end
    data['Province'] = f[:-5]
    province_days.append(data.shape[0])
    province_names.append(f[:-5])
    total_data = pd.concat([total_data,data],axis=0)

In [5]:
total_data.isnull().any()

confirmedCount           False
confirmedIncr            False
curedCount               False
curedIncr                False
currentConfirmedCount    False
currentConfirmedIncr     False
dateId                   False
deadCount                False
deadIncr                 False
highDangerCount          False
midDangerCount           False
suspectedCount           False
suspectedCountIncr       False
Province                 False
dtype: bool

In [6]:
total_data = pd.merge(total_data, level, on='Province', how='left')

In [7]:
total_data = pd.merge(total_data, gdp, on='Province', how='left')

In [8]:
total_data = pd.merge(total_data, population_density, on='Province', how='left')

In [9]:
total_data = pd.merge(total_data, population7th, on='Province', how='left')

In [10]:
total_data = pd.merge(total_data, aging_rate_2021, on='Province', how='left')

In [11]:
vaccinate = vaccinate.iloc[:,1:]

In [12]:
vaccinate['time'] = pd.to_datetime(vaccinate['time'], format='%Y-%m-%d')

In [13]:
vaccinate

,time,vaccination number (w)
0,2021-03-23,8284.60
1,2021-03-24,8585.97
2,2021-03-25,9134.60
3,2021-03-26,9747.00
4,2021-03-27,10241.70
...,...,...
237,2022-05-11,335712.00
238,2022-05-12,335857.60
239,2022-05-13,336005.00
240,2022-05-14,336133.30


In [14]:
total_data

,confirmedCount,confirmedIncr,curedCount,curedIncr,currentConfirmedCount,currentConfirmedIncr,dateId,deadCount,deadIncr,highDangerCount,midDangerCount,suspectedCount,suspectedCountIncr,Province,level,GDP,population_km,population,rate
0,9,9,0,0,9,9,2020-01-23,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0
1,21,12,0,0,21,12,2020-01-24,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0
2,39,18,0,0,39,18,2020-01-25,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0
3,63,24,0,0,63,24,2020-01-26,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0
4,87,24,0,0,87,24,2020-01-27,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29003,1383,0,1354,6,28,-6,2022-05-11,1,0,0,0,0,0,江西,4,29619.7,247.0,45188635,12.3
29004,1383,0,1361,7,21,-7,2022-05-12,1,0,0,0,0,0,江西,4,29619.7,247.0,45188635,12.3
29005,1383,0,1364,3,18,-3,2022-05-13,1,0,0,0,0,0,江西,4,29619.7,247.0,45188635,12.3
29006,1383,0,1369,5,13,-5,2022-05-14,1,0,0,0,0,0,江西,4,29619.7,247.0,45188635,12.3


In [15]:
total_data = pd.merge(total_data, vaccinate, left_on='dateId',right_on='time', how='left').drop(columns=['time']).fillna(method='bfill', limit=10).fillna(0)

In [16]:
total_data

,confirmedCount,confirmedIncr,curedCount,curedIncr,currentConfirmedCount,currentConfirmedIncr,dateId,deadCount,deadIncr,highDangerCount,midDangerCount,suspectedCount,suspectedCountIncr,Province,level,GDP,population_km,population,rate,vaccination number (w)
0,9,9,0,0,9,9,2020-01-23,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0,0.0
1,21,12,0,0,21,12,2020-01-24,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0,0.0
2,39,18,0,0,39,18,2020-01-25,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0,0.0
3,63,24,0,0,63,24,2020-01-26,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0,0.0
4,87,24,0,0,87,24,2020-01-27,0,0,0,0,0,0,山东,4,83095.9,579.0,101527453,16.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29038,1383,0,1354,6,28,-6,2022-05-11,1,0,0,0,0,0,江西,4,29619.7,247.0,45188635,12.3,335712.0
29039,1383,0,1361,7,21,-7,2022-05-12,1,0,0,0,0,0,江西,4,29619.7,247.0,45188635,12.3,335857.6
29040,1383,0,1364,3,18,-3,2022-05-13,1,0,0,0,0,0,江西,4,29619.7,247.0,45188635,12.3,336005.0
29041,1383,0,1369,5,13,-5,2022-05-14,1,0,0,0,0,0,江西,4,29619.7,247.0,45188635,12.3,336133.3


In [17]:
# Data cleaning now end, start to train module

In [18]:
province_days

[828,
 831,
 829,
 829,
 828,
 828,
 829,
 829,
 829,
 829,
 831,
 830,
 828,
 829,
 831,
 829,
 826,
 830,
 828,
 828,
 828,
 827,
 832,
 829,
 829,
 828,
 822,
 829,
 828,
 830,
 829,
 828,
 830,
 830]

In [19]:
# province_days diff in province may means that some days missing!!! In there, just ignore...

In [20]:
groups = total_data.groupby(total_data.Province)
province_data = []
for i in province_names:
    province_data.append(groups.get_group(i))
# province data get end

In [21]:
step1_data = []
for df in province_data:
    df_1 = df.shift(1)
    df_1.rename(columns=lambda x:str(x)+"1", inplace=True)
    step1_data.append(pd.concat([df, df_1], axis=1)[1:])

In [22]:
data_x_y = pd.concat(step1_data, axis=0)

In [23]:
data_x_y

,confirmedCount,confirmedIncr,curedCount,curedIncr,currentConfirmedCount,currentConfirmedIncr,dateId,deadCount,deadIncr,highDangerCount,...,midDangerCount1,suspectedCount1,suspectedCountIncr1,Province1,level1,GDP1,population_km1,population1,rate1,vaccination number (w)1
1,21,12,0,0,21,12,2020-01-24,0,0,0,...,0.0,0.0,0.0,山东,4.0,83095.9,579.0,101527453.0,16.0,0.0
2,39,18,0,0,39,18,2020-01-25,0,0,0,...,0.0,0.0,0.0,山东,4.0,83095.9,579.0,101527453.0,16.0,0.0
3,63,24,0,0,63,24,2020-01-26,0,0,0,...,0.0,0.0,0.0,山东,4.0,83095.9,579.0,101527453.0,16.0,0.0
4,87,24,0,0,87,24,2020-01-27,0,0,0,...,0.0,0.0,0.0,山东,4.0,83095.9,579.0,101527453.0,16.0,0.0
5,121,34,0,0,121,34,2020-01-28,0,0,0,...,0.0,0.0,0.0,山东,4.0,83095.9,579.0,101527453.0,16.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29038,1383,0,1354,6,28,-6,2022-05-11,1,0,0,...,0.0,0.0,0.0,江西,4.0,29619.7,247.0,45188635.0,12.3,335553.5
29039,1383,0,1361,7,21,-7,2022-05-12,1,0,0,...,0.0,0.0,0.0,江西,4.0,29619.7,247.0,45188635.0,12.3,335712.0
29040,1383,0,1364,3,18,-3,2022-05-13,1,0,0,...,0.0,0.0,0.0,江西,4.0,29619.7,247.0,45188635.0,12.3,335857.6
29041,1383,0,1369,5,13,-5,2022-05-14,1,0,0,...,0.0,0.0,0.0,江西,4.0,29619.7,247.0,45188635.0,12.3,336005.0


In [24]:
Y = data_x_y['currentConfirmedCount']
Y_dead = data_x_y['deadCount']
Y_cur = data_x_y['curedCount']
X = data_x_y[['curedCount1','curedIncr1','currentConfirmedCount1','currentConfirmedIncr1','deadCount1','deadIncr1','level1','GDP1','population_km1','population1','rate1']]

In [25]:
X = X.reset_index(drop=True)
Y = Y.reset_index(drop=True)
Y_dead = Y_dead.reset_index(drop=True)
Y_cur = Y_cur.reset_index(drop=True)

In [26]:
X

,curedCount1,curedIncr1,currentConfirmedCount1,currentConfirmedIncr1,deadCount1,deadIncr1,level1,GDP1,population_km1,population1,rate1
0,0.0,0.0,9.0,9.0,0.0,0.0,4.0,83095.9,579.0,101527453.0,16.0
1,0.0,0.0,21.0,12.0,0.0,0.0,4.0,83095.9,579.0,101527453.0,16.0
2,0.0,0.0,39.0,18.0,0.0,0.0,4.0,83095.9,579.0,101527453.0,16.0
3,0.0,0.0,63.0,24.0,0.0,0.0,4.0,83095.9,579.0,101527453.0,16.0
4,0.0,0.0,87.0,24.0,0.0,0.0,4.0,83095.9,579.0,101527453.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
29004,1348.0,16.0,34.0,-16.0,1.0,0.0,4.0,29619.7,247.0,45188635.0,12.3
29005,1354.0,6.0,28.0,-6.0,1.0,0.0,4.0,29619.7,247.0,45188635.0,12.3
29006,1361.0,7.0,21.0,-7.0,1.0,0.0,4.0,29619.7,247.0,45188635.0,12.3
29007,1364.0,3.0,18.0,-3.0,1.0,0.0,4.0,29619.7,247.0,45188635.0,12.3


In [27]:
train_X,test_X,train_y,test_y = train_test_split(X,Y,test_size=0.3, random_state=4)
train_X_dead,test_X_dead,train_y_dead,test_y_dead = train_test_split(X,Y_dead,test_size=0.3, random_state=4)
train_X_cur,test_X_cur,train_y_cur,test_y_cur = train_test_split(X,Y_cur,test_size=0.3, random_state=4)

In [28]:
le = LabelEncoder()
train_X = train_X.reset_index(drop=True)
test_X = test_X.reset_index(drop=True)
train_y = le.fit_transform(train_y)
test_y = le.fit_transform(test_y)

train_X_dead = train_X_dead.reset_index(drop=True)
test_X_dead = test_X_dead.reset_index(drop=True)
train_y_dead = le.fit_transform(train_y_dead)
test_y_dead = le.fit_transform(test_y_dead)

train_X_cur = train_X_cur.reset_index(drop=True)
test_X_cur = test_X_cur.reset_index(drop=True)
train_y_cur = le.fit_transform(train_y_cur)
test_y_cur = le.fit_transform(test_y_cur)

In [29]:
# model start
# xgboost
xgb_params = {
    'random_state': 1, 
    'n_jobs': 8,
    'booster': 'gbtree',
    'n_estimators': 10000,
    'tree_method': 'gpu_hist', 
    'gpu_id': 0,
    # optimized params
    'learning_rate': 0.03628302216953097,
    'reg_lambda': 0.0008746338866473539,
    'reg_alpha': 23.13181079976304,
    'subsample': 0.7875490025178415,
    'colsample_bytree': 0.11807135201147481,
    'max_depth': 10
}

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'learning_rate': [0.05,0.1,0.03628302216953097], #so called `eta` value
              'max_depth': [6,7,8,9,10],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8,0.7875490025178415],
              'colsample_bytree': [0.7,0.11807135201147481],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337],
             'gpu_id': [0],
            'n_jobs': [8],
            'booster': ['gbtree'],
            'tree_method': ['gpu_hist'], 
            'gpu_id': [0],
            'reg_lambda': [0.0008746338866473539],
            'reg_alpha': [23.13181079976304],
             }

# clf = XGBRegressor(**xgb_params)

clf = GridSearchCV(XGBRegressor(), parameters, n_jobs=5, 
                   cv=5,
                   scoring='roc_auc',
                   verbose=100, refit=True)

# clf = MLPRegressor(
#   hidden_layer_sizes=(1000,750,500,250,100,50),
#   activation='relu',
# #     solver='sgd',
#     alpha=0.01,
#     max_iter=1000,
#     early_stopping=True,
#     verbose=True,
# #     learning_rate_init=0.01
# )

In [30]:
clf.fit(train_X,
        train_y,
#         eval_set=[(test_X,test_y)],
#         early_stopping_rounds=300,
#         verbose = 100
       )

Fitting 5 folds for each of 60 candidates, totalling 300 fits


/home/bill/.local/lib/python3.9/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(


[19:52:50] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat...
                         'learning_rate': [0.05, 0.1, 0.03628302216953097],
                         'max_depth': [6, 7, 8, 9, 10],
                         'min_child_weight': [11], 'missing': [-999],
                         'n_estimators': [1000], 'n_jo

In [31]:
plt.plot(clf.predict(test_X))
plt.plot(test_y)

<IPython.core.display.Javascript object>

In [32]:
sum = 0
count = 0
for i in test_y - clf.predict(test_X):
    sum += abs(i)
    count += 1
for i in train_y - clf.predict(train_X):
    sum += abs(i)
    count += 1
print(sum / count)

4.625974246779412


In [33]:
clf.cv_results_

{'mean_fit_time': array([21.21543612, 21.4715611 , 29.32527356, 29.20710959, 40.25310278,
        44.93474317, 52.44927182, 52.96063275, 66.84046459, 63.78076229,
        23.79420915, 24.22775245, 33.49062934, 31.83817058, 42.72539191,
        41.85047407, 52.93555703, 52.7423758 , 64.78065958, 63.21123562,
        22.17885303, 23.51591258, 31.85839353, 31.8443912 , 44.76357298,
        43.40002961, 51.96971312, 50.51594734, 60.90997438, 60.18174024,
        14.23687434, 13.35525417, 16.13699293, 15.82315555, 18.15894122,
        19.2754858 , 20.68170843, 21.00876622, 22.10229607, 21.83666797,
        13.82929769, 13.69301534, 16.03032546, 16.17874193, 18.20609126,
        18.33456593, 20.05362701, 20.54864378, 21.7251698 , 21.58535852,
        14.91837692, 14.68373995, 17.35652928, 17.44204946, 19.68839974,
        19.75170903, 21.60187483, 21.62761178, 23.1060205 , 19.96534457]),
 'std_fit_time': array([0.07676912, 0.29148534, 0.32453003, 0.40290724, 0.58310009,
        0.79537639, 0

In [34]:
clf.best_params_

{'booster': 'gbtree',
 'colsample_bytree': 0.7,
 'gpu_id': 0,
 'learning_rate': 0.05,
 'max_depth': 6,
 'min_child_weight': 11,
 'missing': -999,
 'n_estimators': 1000,
 'n_jobs': 8,
 'nthread': 4,
 'reg_alpha': 23.13181079976304,
 'reg_lambda': 0.0008746338866473539,
 'seed': 1337,
 'silent': 1,
 'subsample': 0.8,
 'tree_method': 'gpu_hist'}

In [35]:
clf.predict(test_X)

array([3.9917543e+00, 1.1868752e+01, 4.7693699e+01, ..., 5.9515409e-02,
       6.2533012e+01, 4.5516911e+00], dtype=float32)

In [36]:
test_y

array([ 4, 10, 48, ...,  0, 63,  4])

In [37]:
clf.fit(train_X_dead,
        train_y_dead,
#         eval_set=[(test_X_dead,test_y_dead)],
#         early_stopping_rounds=300,
#         verbose = 100
       )
plt.figure()
plt.plot(clf.predict(test_X_dead))
plt.plot(test_y_dead)

sum = 0
count = 0
for i in test_y_dead - clf.predict(test_X_dead):
    sum += abs(i)
    count += 1
for i in train_y_dead-clf.predict(train_X_dead):
    sum += abs(i)
    count += 1
print(sum / count)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


KeyboardInterrupt: 

In [ ]:
clf.fit(train_X_cur,
        train_y_cur,
#         eval_set=[(test_X_cur,test_y_cur)],
#         early_stopping_rounds=300,
#         verbose = 100
       )
plt.figure()
plt.plot(clf.predict(test_X_cur))
plt.plot(test_y_cur)

sum = 0
count = 0
for i in test_y_cur - clf.predict(test_X_cur):
    sum += abs(i)
    count += 1
for i in train_y_cur-clf.predict(train_X_cur):
    sum += abs(i)
    count += 1
print(sum / count)

In [ ]:
plt.plot(clf.predict(train_X))

In [ ]:
plt.plot(train_y)

In [ ]:
abs(train_y - clf.predict(train_X)).mean()